## Java 虚拟机中虚方法调用的具体实现
- demo 案例代码

```
abstract class Passenger {
  abstract void passThroughImmigration();
  @Override
  public String toString() { ... }
}
class ForeignerPassenger extends Passenger {
   @Override
   void passThroughImmigration() { /* 进外国人通道 */ }
}
class ChinesePassenger extends Passenger {
  @Override
  void passThroughImmigration() { /* 进中国人通道 */ }
  void visitDutyFreeShops() { /* 逛免税店 */ }
}

Passenger passenger = ...
passenger.passThroughImmigration();
```

### 1. 虚方法调用
- 动态绑定
  - Java 虚拟机需要根据调用者的动态类型，来确定虚方法调用的目标方法
  - 非私有实例方法: **invokevirtual 指令**
  - 接口方法: **invokeinterface 指令**
- 静态绑定
  - 静态方法: **invokestatic 指令**
  - 构造器、私有实例方法以及超类非私有实例方法: **invokespecial 指令**
  - 虚方法调用指向标记为 final 的方法: 虚拟机也可以**静态绑定**该虚方法调用的目标方法

### 2. 方法表
- **空间换取时间**的策略来实现动态绑定
  - 为每个类生成一张方法表，用以快速定位目标方法
- 类加载链接过程中的准备阶段
  - 构造与该类相关联的方法表

#### invokevirtual 案例
- invokeinterface 稍微复杂些，但原理是类似的
- 方法表本质上是一个数组，每个数组元素指向一个**当前类**及其**祖先类**中非私有的实例方法
  - 可能是具体的方法 或者 抽象方法，**方法表**满足如下特质
    - 子类方法表中包含父类方法表中的所有方法
    - 子类方法在方法表中的索引值，与它所**重写的父类方法**的索引值相同

![方法表](方法表.png)

- 为了简洁，这里不考虑 Object 类中的其他方法
- 举例：虚拟机类似于出境导航员，根据游客国籍，找到对应的方法表，根据相关索引，调用对应的方法
- 动态绑定与静态绑定相比: 多了几个内存解析引用操作
  - 访问栈上的调用者，读取动态类型，读取该类型的方法表，读取方法表中某个索引值对应的目标方法
- 更好的性能优化方案
  - **内联缓存**
  - **方法内联**

### 3. 内联缓存
- 内联缓存是一种加快动态绑定的优化技术，能够缓存虚方法调用中调用者的动态类型，以及该类型所对应的目标方法
  - 已缓存的类型: 内联缓存便会直接调用该类型所对应的目标方法
  - 没有缓存的类型: 内联缓存则会退化至使用基于方法表的动态绑定

#### 多态的优化手段
- 单态(monomorphic): 仅有一种状态的情况
- 多态(polymorphic): 有限数量种状态的情况(二态是多态的一种)
- 超多态(megamorphic): 更多种状态的情况(通常用一个具体数值来区分多态和超多态)

#### 内联缓存分类
- 单态内联缓存
  - 在实践中，大部分的虚方法调用均是单态的，也就是只有一种动态类型
  - 为了节省内存空间，Java 虚拟机只采用**单态内联缓存**
- 多态内联缓存
- 超多态内联缓存

#### 内联缓存没有命中的情况下
- 两种选择
  - 替换单态内联缓存中的纪录
    - 最坏情况: 两种类型交替访问，导致只有写缓存的额外开销，而没有用缓存的性能提升
  - 劣化为超多态状态
    - **Java 虚拟机的具体实现方式**
    - 它将直接访问方法表，来动态绑定目标方法（放弃了优化的机会）

#### 其他
- 内联缓存并没有**内联目标方法**
- **任何方法调用除非被内联，否则都会有固定开销**
  - 开销来源于保存程序在该方法中的执行位置，以及新建、压入和弹出新方法所使用的栈帧
- 在即时编译中，方法内联不仅仅能够消除方法调用的固定开销，而且还增加了进一步优化的可能性
  - 专栏的第二部分详细介绍方法内联的内容

---

## 课后实践
- 观测一下单态内联缓存和超多态内联缓存的性能差距
- 为了消除方法内联的影响，请使用如下的命令

```
// Run with: java -XX:CompileCommand='dontinline,*.passThroughImmigration' Passenger
public abstract class Passenger {
   abstract void passThroughImmigration();
  public static void main(String[] args) {
    Passenger a = new ChinesePassenger();
  Passenger b = new ForeignerPassenger();
    long current = System.currentTimeMillis();
    for (int i = 1; i <= 2_000_000_000; i++) {
      if (i % 100_000_000 == 0) {
        long temp = System.currentTimeMillis();
        System.out.println(temp - current);
        current = temp;
      }
      Passenger c = (i < 1_000_000_000) ? a : b;
      c.passThroughImmigration();
  }
  }
}
class ChinesePassenger extends Passenger {
  @Override void passThroughImmigration() {} 
}
class ForeignerPassenger extends Passenger {
  @Override void passThroughImmigration() {}
}
```

---